# Module E: Time Evolution and Pictures

## The Schrödinger Equation

$$i\hbar\frac{d|\psi(t)\rangle}{dt} = \hat{H}|\psi(t)\rangle$$

### Solution

$$|\psi(t)\rangle = e^{-i\hat{H}t/\hbar}|\psi(0)\rangle = \hat{U}(t)|\psi(0)\rangle$$

The **time evolution operator** $\hat{U}(t) = e^{-i\hat{H}t/\hbar}$ is unitary.

## Three Pictures of QM

| Picture | States | Operators | Expectation |
|---------|--------|-----------|-------------|
| Schrödinger | Evolve | Fixed | ⟨ψ(t)|A|ψ(t)⟩ |
| Heisenberg | Fixed | Evolve | ⟨ψ|A(t)|ψ⟩ |
| Interaction | Both evolve | Both evolve | Mixed |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML
from scipy.linalg import expm

plt.style.use('seaborn-v0_8-whitegrid')

In [ ]:
# Helper functions
def normalize(psi):
    return psi / np.sqrt(np.vdot(psi, psi))

def expectation_value(psi, A):
    return np.real(np.vdot(psi, A @ psi))

# Pauli matrices
I = np.eye(2, dtype=complex)
sigma_x = np.array([[0, 1], [1, 0]], dtype=complex)
sigma_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
sigma_z = np.array([[1, 0], [0, -1]], dtype=complex)

## 2-Level System Hamiltonian

Consider a spin-1/2 particle in a magnetic field:

$$\hat{H} = \frac{\omega_0}{2}\sigma_z + \frac{\Omega}{2}\sigma_x$$

- $\omega_0$: energy splitting between levels
- $\Omega$: coupling between levels (e.g., RF field)

### Physical Picture

The spin precesses around an effective field direction.

In [ ]:
def create_hamiltonian(omega_0, Omega):
    """
    Create 2-level Hamiltonian H = (ω₀/2)σz + (Ω/2)σx
    
    Parameters:
    -----------
    omega_0 : float
        Level splitting (ℏ = 1 units)
    Omega : float
        Coupling strength (Rabi frequency)
    """
    return (omega_0/2) * sigma_z + (Omega/2) * sigma_x

# Example parameters
omega_0 = 1.0  # Level splitting
Omega = 0.5    # Coupling
H = create_hamiltonian(omega_0, Omega)

print("Hamiltonian H:")
print(H)

# Find eigenvalues and eigenstates
eigenvalues, eigenstates = np.linalg.eigh(H)
print(f"\nEnergies: {eigenvalues}")

## Schrödinger Picture Evolution

$$|\psi(t)\rangle = e^{-iHt}|\psi(0)\rangle$$

We use `scipy.linalg.expm` for the matrix exponential.

In [ ]:
def schrodinger_evolve(psi0, H, t):
    """
    Evolve state in Schrödinger picture.
    
    |ψ(t)⟩ = exp(-iHt)|ψ(0)⟩  (ℏ = 1)
    """
    U = expm(-1j * H * t)  # Time evolution operator
    return U @ psi0

## Heisenberg Picture Evolution

$$\hat{A}_H(t) = e^{iHt}\hat{A}e^{-iHt}$$

Operators evolve, states stay fixed.

In [ ]:
def heisenberg_evolve_operator(A, H, t):
    """
    Evolve operator in Heisenberg picture.
    
    A_H(t) = exp(iHt) A exp(-iHt)
    """
    U = expm(-1j * H * t)
    U_dag = np.conj(U.T)
    return U_dag @ A @ U

In [ ]:
# ============================================================
# VERIFY: ⟨A⟩ same in both pictures
# ============================================================

psi0 = np.array([1, 0], dtype=complex)  # Start in |↑⟩
t = 2.0

# Schrödinger picture
psi_t = schrodinger_evolve(psi0, H, t)
exp_schrodinger = expectation_value(psi_t, sigma_z)

# Heisenberg picture
sigma_z_H = heisenberg_evolve_operator(sigma_z, H, t)
exp_heisenberg = expectation_value(psi0, sigma_z_H)

print("Expectation Value Comparison at t = 2.0:")
print("="*45)
print(f"Schrödinger: ⟨ψ(t)|σz|ψ(t)⟩ = {exp_schrodinger:.6f}")
print(f"Heisenberg:  ⟨ψ₀|σz(t)|ψ₀⟩  = {exp_heisenberg:.6f}")
print(f"\n✓ Agreement: {np.isclose(exp_schrodinger, exp_heisenberg)}")

## Time Evolution of Expectation Values

For the 2-level system, we can track:
- $\langle\sigma_x(t)\rangle$
- $\langle\sigma_y(t)\rangle$  
- $\langle\sigma_z(t)\rangle$

These trace out the **Bloch vector** trajectory.

In [ ]:
# Compute expectation values over time
times = np.linspace(0, 4 * np.pi, 200)
psi0 = np.array([1, 0], dtype=complex)

exp_x, exp_y, exp_z = [], [], []

for t in times:
    psi_t = schrodinger_evolve(psi0, H, t)
    exp_x.append(expectation_value(psi_t, sigma_x))
    exp_y.append(expectation_value(psi_t, sigma_y))
    exp_z.append(expectation_value(psi_t, sigma_z))

exp_x, exp_y, exp_z = np.array(exp_x), np.array(exp_y), np.array(exp_z)

In [ ]:
# Plot expectation values vs time
fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(times, exp_x, label='⟨σx⟩', linewidth=2)
ax.plot(times, exp_y, label='⟨σy⟩', linewidth=2)
ax.plot(times, exp_z, label='⟨σz⟩', linewidth=2)

ax.set_xlabel('Time')
ax.set_ylabel('Expectation Value')
ax.set_title('Spin Precession')
ax.legend(loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_ylim(-1.1, 1.1)

plt.tight_layout()
plt.savefig('spin_precession.png', dpi=150)
plt.show()

## The Bloch Sphere

### Representation

Any pure state of a 2-level system can be written:
$$|\psi\rangle = \cos(\theta/2)|0\rangle + e^{i\phi}\sin(\theta/2)|1\rangle$$

This maps to a point $(x, y, z)$ on the unit sphere:
- $x = \langle\sigma_x\rangle$
- $y = \langle\sigma_y\rangle$
- $z = \langle\sigma_z\rangle$

### Physical Interpretation
- North pole: $|\uparrow\rangle$
- South pole: $|\downarrow\rangle$
- Equator: superpositions

In [ ]:
def bloch_vector(psi):
    """Compute Bloch vector (x, y, z) from state."""
    x = expectation_value(psi, sigma_x)
    y = expectation_value(psi, sigma_y)
    z = expectation_value(psi, sigma_z)
    return x, y, z

In [ ]:
# ============================================================
# 3D Bloch sphere trajectory
# ============================================================

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Draw sphere wireframe
u = np.linspace(0, 2 * np.pi, 30)
v = np.linspace(0, np.pi, 20)
x_sphere = np.outer(np.cos(u), np.sin(v))
y_sphere = np.outer(np.sin(u), np.sin(v))
z_sphere = np.outer(np.ones(np.size(u)), np.cos(v))
ax.plot_wireframe(x_sphere, y_sphere, z_sphere, alpha=0.1, color='gray')

# Draw trajectory
ax.plot(exp_x, exp_y, exp_z, 'b-', linewidth=2, label='Trajectory')
ax.scatter([exp_x[0]], [exp_y[0]], [exp_z[0]], color='green', s=100, label='Start')
ax.scatter([exp_x[-1]], [exp_y[-1]], [exp_z[-1]], color='red', s=100, label='End')

# Axes
ax.quiver(0, 0, 0, 1.3, 0, 0, color='red', arrow_length_ratio=0.1)
ax.quiver(0, 0, 0, 0, 1.3, 0, color='green', arrow_length_ratio=0.1)
ax.quiver(0, 0, 0, 0, 0, 1.3, color='blue', arrow_length_ratio=0.1)
ax.text(1.4, 0, 0, 'x', fontsize=12)
ax.text(0, 1.4, 0, 'y', fontsize=12)
ax.text(0, 0, 1.4, 'z', fontsize=12)

ax.set_xlim([-1.2, 1.2])
ax.set_ylim([-1.2, 1.2])
ax.set_zlim([-1.2, 1.2])
ax.set_title('Bloch Sphere: Spin Precession Trajectory')
ax.legend()

plt.tight_layout()
plt.savefig('bloch_sphere.png', dpi=150)
plt.show()

In [ ]:
# ============================================================
# ANIMATION: Bloch vector precession
# ============================================================

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')

def init():
    ax.set_xlim([-1.2, 1.2])
    ax.set_ylim([-1.2, 1.2])
    ax.set_zlim([-1.2, 1.2])
    return []

def animate(frame):
    ax.clear()
    
    # Sphere
    ax.plot_wireframe(x_sphere, y_sphere, z_sphere, alpha=0.1, color='gray')
    
    # Trajectory up to current frame
    ax.plot(exp_x[:frame+1], exp_y[:frame+1], exp_z[:frame+1], 'b-', lw=1.5, alpha=0.6)
    
    # Current Bloch vector
    ax.quiver(0, 0, 0, exp_x[frame], exp_y[frame], exp_z[frame],
              color='red', arrow_length_ratio=0.15, linewidth=3)
    
    # Axes labels
    ax.text(1.3, 0, 0, 'x')
    ax.text(0, 1.3, 0, 'y')
    ax.text(0, 0, 1.3, 'z')
    
    ax.set_xlim([-1.2, 1.2])
    ax.set_ylim([-1.2, 1.2])
    ax.set_zlim([-1.2, 1.2])
    ax.set_title(f'Bloch Vector (t = {times[frame]:.2f})')
    
    return []

anim = FuncAnimation(fig, animate, init_func=init, frames=range(0, len(times), 2),
                     interval=50, blit=False)
plt.close()
HTML(anim.to_jshtml())

## Summary

### Time Evolution

| Picture | Who Evolves | Formula |
|---------|-------------|----------|
| Schrödinger | States | $\|\psi(t)\rangle = e^{-iHt}\|\psi(0)\rangle$ |
| Heisenberg | Operators | $A(t) = e^{iHt}Ae^{-iHt}$ |

### Key Results

1. Expectation values are the same in both pictures
2. 2-level systems map to the Bloch sphere
3. Time evolution → precession on the sphere
4. The Hamiltonian determines the precession axis